In [138]:
#import libraries
# import tensorflow as tf
import torch
from torch.autograd import Variable
# torch.__version__

#Task 1: Stochastic Gradient Descent

##Implement a polynomial function polynomial_fun, that takes two input arguments, a weight vector w of size M+1 and an input scalar variable x, and returns the function value y

In [109]:
def polynomial_fun(w,x):
    '''
    Inputs:
    w = tensor of weights
    x = tesnor of scalar values of x, our r.v.

    Outpus:
    y = tensor of the sum of our polynomial fit
    '''
    
    m = len(w)
    n = len(x)

    basis = Variable(torch.zeros([n,m]))

    for i in range(m):
        basis[:,i] = torch.pow(x,i)

    y = torch.sum(w*basis,1)   

    return y

##Using linear algebra modules in Tensorflow/PyTorch implement a least square solver for fitting the polyomial functions, fit_polynoial_ls, which takes N pairs of x and target values t as input, with an additional input argument to specify the polynomial degree M, and returns the optimum weight vector in least square sense.

In [174]:
def fit_polynomial_ls(X,t,M):
    '''
    Inputs:
    t = tensor of target value
    X = tensor of scalar values of x, our r.v.
    M = how many powers in our polynomial fit

    Outpus:
    w = tensor of weight values
    '''

    #initialize empty vector to store feature map of basis functions
    n = len(X)
    basis = Variable(torch.zeros([n,M]))

    for i in range(M):
        basis[:,i] = torch.pow(X,i)

    #solve for weights: w = (X.T * X)^-1 *X.T*y
    A = torch.matmul(basis.T, basis)
    b = torch.matmul(basis.T, t)

    w = torch.matmul(torch.linalg.torch.inverse(A),b)

    return w


##Implement a stochastic minibatch gradient descent algorithm for fitting polynomial functions which has the same input as fit_polynomial_ls with additional 2 input arguments: learning rate and minibatch size. The function also returns the optimum w and during training should report the loss periodically using printed messages.

In [ ]:
def fit_polynomial_sgd(X,t,M, learn_rate, batch_size):
    

###task script:

#### 
Use polynomial_fun(M=3,w=(1,2,3,4)) to generate a training set and a test set in the form of respectively sampled 100 and 50 pairs x, x in [20,20] and observed t. The observed t values are obtained by adding gaussian noise, std = .2 to y

In [179]:
w = torch.tensor([1,2,3,4])

x_train = torch.distributions.uniform.Uniform(-20,20).sample([100,])
x_test =  torch.distributions.uniform.Uniform(-20,20).sample([50,])

y_train = polynomial_fun(w,x_train) + torch.normal(0,.2, size = x_train.shape)
y_test = polynomial_fun(w,x_test) + torch.normal(0,.2,size = x_test.shape)

####
Use fit_polynomial_ls(M=4) to compute the optimum weight vector w using the trianing set. Then predict target values for trianing and test set.

In [182]:

w_hat = fit_polynomial_ls(x_train,y_train,4)
y_train_hat = polynomial_fun(x_train,w_hat)
y_test_hat = polynomial_fun(x_test,w_hat)

####
Report using printed messages the meand and std in difference betwen A) the observed training data and the underlying true polynomial curve and B) the predicted values and the true polynomial curve